In [ ]:
import csv
import random
import sqlite3
import pandas as pd

In [ ]:
# 데이터베이스에서 데이터 불러오기 (SQLite3 사용)
def fetch_data_from_db(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # 강의와 키워드를 연결하는 테이블에서 데이터를 가져옴
    cursor.execute("""
        SELECT l.lecture_title, k.keyword 
        FROM STT_lecture l
        JOIN STT_lecture_keywords lk ON l.id = lk.lecture_id
        JOIN STT_keywords k ON lk.keyword_id = k.id
    """)
    lecture_keywords = cursor.fetchall()

    conn.close()

    return lecture_keywords


In [ ]:
# 자모음 랜덤 추가와 자모음 변경을 모두 포함하는 오타 생성 함수
def generate_typo_jamo_v2(word):
    if len(word) > 1:
        typo_type = random.choice(["vowel_change", "vowel_add", "consonant_change", "consonant_add"])
        vowels = "ㅏㅑㅓㅕㅗㅛㅜㅠㅡㅣ"
        consonants = "ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎ"
        if typo_type == "vowel_change":
            # 모음을 변경
            return word.replace(word[1], random.choice(vowels))
        elif typo_type == "vowel_add":
            # 모음을 추가
            return word[:1] + random.choice(vowels) + word[1:]
        elif typo_type == "consonant_change":
            # 자음을 변경
            return word.replace(word[0], random.choice(consonants))
        elif typo_type == "consonant_add":
            # 자음을 추가
            return random.choice(consonants) + word
    else:
        return word  # 한 글자 키워드는 그대로 반환

In [ ]:
# CSV 파일 생성 함수
def generate_test_dataset(csv_file_path, lectures, keywords, num_questions=100):
    with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Question', 'Expected Answer'])

        # 질문 유형 정의
        question_types = ["단순 질문", "오타가 포함된 질문", "복합 질문", "아이들이 할 법한 질문", "구체적 설명 요청"]
        num_questions_per_type = num_questions // len(question_types)  # 각 질문 유형에 대해 동일한 수의 질문을 생성

        # 질문 형식들을 다양화하여 각 질문 유형에 여러 질문을 반영
        # 조사 및 오타 포함 질문은 부분적으로 수기로 수정함
        question_templates = {
            "단순 질문": [
                "{keyword}에 대해 알려줘",
                "{keyword} 강의 어디서 볼 수 있어?",
                "{keyword} 강의 설명해 줘"
            ],
            "오타가 포함된 질문": [
                "{typo_keyword} 관련 자료 있어?",
                "{typo_keyword} 내용 좀 알려 줄래?",
                "{typo_keyword} 강의 찾고 싶어"
            ],
            "복합 질문": [
                "{keyword}와 {second_keyword} 설명해 줘",
                "{keyword}랑 {second_keyword} 같이 배울 수 있는 강의 있어?",
                "{keyword}하고 {second_keyword}에 대한 강의 듣고 싶어"
            ],
            "아이들이 할 법한 질문": [
                "{keyword}가 뭐에요?",
                "{keyword}는 어떻게 사용하는 거예요?",
                "{keyword}이 너무 어려워요"
            ],
            "구체적 설명 요청": [
                "{keyword} 어떻게 구해?",
                "{keyword} 풀 때 중요한 점은 뭐야?",
                "{keyword}를 더 쉽게 푸는 방법 있어?"
            ]
        }

        # 데이터를 다시 생성
        for q_type in question_types:
            for _ in range(num_questions_per_type):
                # 무작위로 강의와 키워드 선택
                random_lecture = random.choice(lectures)
                random_keyword = random.choice(keywords)

                # 질문 형식 적용
                if q_type == "단순 질문":
                    template = random.choice(question_templates["단순 질문"])
                    question = template.format(keyword=random_keyword)
                    answer = random_lecture
                
                elif q_type == "오타가 포함된 질문":
                    typo_keyword = generate_typo_jamo_v2(random_keyword)  # 자모음을 랜덤하게 변경
                    template = random.choice(question_templates["오타가 포함된 질문"])
                    question = template.format(typo_keyword=typo_keyword)
                    answer = random_lecture
                
                elif q_type == "복합 질문":
                    second_keyword = random.choice(keywords)
                    template = random.choice(question_templates["복합 질문"])
                    question = template.format(keyword=random_keyword, second_keyword=second_keyword)
                    answer = random_lecture
                
                elif q_type == "아이들이 할 법한 질문":
                    template = random.choice(question_templates["아이들이 할 법한 질문"])
                    question = template.format(keyword=random_keyword)
                    answer = random_lecture
                
                elif q_type == "구체적 설명 요청":
                    template = random.choice(question_templates["구체적 설명 요청"])
                    question = template.format(keyword=random_keyword)
                    answer = random_lecture

                # 데이터 CSV 파일에 작성
                writer.writerow([question, answer])

    print(f"{csv_file_path} 파일이 성공적으로 생성되었습니다.")

In [ ]:
# 메인 함수
def main():
    db_path = 'db.sqlite3'  # SQLite3 데이터베이스 경로
    csv_file_path = 'test_dataset_varied_questions_100.csv'

    # 데이터베이스에서 강의와 키워드 데이터 가져오기
    lectures, keywords = fetch_data_from_db(db_path)

    # CSV 파일로 테스트 데이터셋 생성
    generate_test_dataset(csv_file_path, lectures, keywords, num_questions=100)

if __name__ == "__main__":
    main()